# Load Dependencies

In [1]:
import os
import json
import random
from collections import defaultdict
from typing import List, Tuple
from pydantic import BaseModel, computed_field

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph

import pandas as pd
import json

load_dotenv()
os.environ.get("OPENAI_API_KEY")

dataset_name = "contractnli"
test_file = f"../data/benchmarks/{dataset_name}.json"
result_file = f"../data/results/qa_results.json"

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

persist_path = "./vectorstore/chroma_openai_embed_3_small"

vector_store = Chroma(
        embedding_function=embeddings, 
        persist_directory=persist_path
    )
# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(model="gpt-4o-mini")

# Generate response from retrieved context

## Get query and context from JSON

In [2]:
import json
import random

with open("./data/json_output/contractnli.json") as f:
    QnA_data = json.load(f)

In [3]:
import json

print(QnA_data[0].keys())
print(QnA_data[0]['query'])

print(f"number of chunks: {len(QnA_data[0]['retrieved_chunks_unranked'])}")

for chunk in QnA_data[0]['retrieved_chunks_unranked']:
    # print(chunk['text'])
    print(chunk)

dict_keys(['query', 'snippets', 'file_set', 'query_rewriter', 'feature_extraction', 'retrieved_chunks_unranked'])
Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?
number of chunks: 3
{'chunk_id': 21561, 'filepath': 'contractnli/CopAcc_NDA-and-ToP-Mentors_2.0_2017.txt', 'span': [7711, 8160], 'text': '4 Definition of Confidential Information\n“Confidential Information” means any Idea disclosed to Mentor, all data and information, know-how, business concepts, software, procedures, products, services, development projects, and programmes contained in such Idea and/or its description and any conclusions. Confidential Information does not include:\n> information already known or independently developed by Mentor prior to the disclosure of any Idea;'}
{'chunk_id': 21539, 'filepath': 'contractnli/CopAcc_NDA-and-ToP-Mentors_2.0_2017.txt', 'span': [359

In [4]:
print(QnA_data)

[{'query': 'Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?', 'snippets': [{'file_path': 'contractnli/CopAcc_NDA-and-ToP-Mentors_2.0_2017.txt', 'span': [11461, 11963], 'answer': 'Any and all proprietary rights, including but not limited to rights to and in inventions, patent rights, utility models, copyrights, trademarks and trade secrets, in and to any Confidential Information shall be and remain with the Participants respectively, and Mentor shall not have any right, license, title or interest in or to any Confidential Information, except the limited right to review, assess and help develop such Confidential Information in connection with the Copernicus Accelerator 2017.'}], 'file_set': ['CopAcc_NDA-and-ToP-Mentors_2.0_2017.txt'], 'query_rewriter': [{'best_file_path': 'contractnli/CopAcc_NDA-and-ToP-Mentors_2.0_2017.txt', 'file_locator': '

In [5]:
QnA_data_len = len(QnA_data)
print(QnA_data_len)

def get_dataset_from_json(index=0):
    
    qna_data_query = QnA_data[index]["query"]
    retrieved_chunks = QnA_data[index]["retrieved_chunks_unranked"]

    contexts_from_json = []
    for context in retrieved_chunks:
        cur_context = {}
        cur_context["file_path"] = context["filepath"]
        cur_context["span"] = context["span"]
        cur_context["chunk"] = context["text"]
        contexts_from_json.append(cur_context)
    
    return qna_data_query, contexts_from_json

194


## Answer Generation Pipeline

In [6]:
from langchain import hub
from typing_extensions import List, TypedDict
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

class ResponseGeneration:
    class State(TypedDict):
        question : str
        context : List[Document]
        answer: str

    def __init__(self, prompt: PromptTemplate, model = "gpt-4o-mini", temperature = 0.2):
        self.llm = ChatOpenAI(model=model, temperature=temperature)
        self.prompt = prompt

        graph_builder = StateGraph(self.State)
        graph_builder.add_sequence([self.generate])
        graph_builder.add_edge(START, "generate")
        self.graph = graph_builder.compile()

    def generate(self, state : State):
        context_doc_message = "\n\n".join(doc for doc in state["context"])
        message = self.prompt.invoke({"question":state["question"], "context":context_doc_message})
        response = self.llm.invoke(message)

        return({"answer":response})
    

In [62]:
# define baseline prompt
zeroshot_CoT_prompt = PromptTemplate.from_template("""HUMAN\n
                                                You are an assistant for question-answering tasks. Think step by step on each of retrieved context on how they help answer the question. Think how different terminologies or party names or entity names in the contexts are related to the question. Think if all the contexts is relevant to the question and ONLY use relevant information to answer the question.\n
                                                How to write the final answer: Reference and clear up different terminologies or party name between the question and contexts to clear up any confusion. State facts that imply something that answer the question. Use maximum of two sentences in the final answer. Just say you don't know if you cannot generate meaningful and factual answer\n
                                                Question: {question}\n
                                                Context: {context}\n
                                                Step-by-step reasoning: Let's think step by step on the context.\n
                                                Answer formatting: JSON format {{"thought":"AI thought","answer":"The actual answer"}}
                                                Answer: 
                                                """)

In [45]:
# define baseline prompt
zeroshot_CoT_prompt = PromptTemplate.from_template("""HUMAN\n
                                                You are a world class assistant for legal case question-answering tasks. Think step by step on each of retrieved context on how they help answer the question. Think how different terminologies or party names or entity names in the contexts are related to the question. Think if all the contexts is relevant to the question and ONLY use relevant information to answer the question.\n
                                                How to write the final answer: Answer in legal counseling manner: clear up different terminologies or party name between the question and contexts by mentioning the equivalent terms/word/entity/name in the final answer. State ONLY facts and explicitly say if it ONLY IMPLIES something that answer the question in the final answer. Use maximum of three sentences in the final answer. Just say you don't know if you cannot generate meaningful and factual answer\n
                                                Question: {question}\n
                                                Context: {context}\n
                                                Step-by-step reasoning: Let's think step by step on the context.\n
                                                Answer formatting: JSON format {{"thought":"AI thought","answer":"The actual answer"}}
                                                Answer: 
                                                """)

In [63]:
# baseline response test
zeroshot_CoT_response_generation = ResponseGeneration(zeroshot_CoT_prompt)

query, contexts = get_dataset_from_json(index=193)
output = zeroshot_CoT_response_generation.graph.invoke({"question":query, "context":[context["chunk"] for context in contexts]})

In [64]:
print(query)
print("\n".join([context["chunk"] for context in contexts]))
print(output["answer"])


Consider the Mutual Non-Disclosure Agreement between IPTK and CO; Does the document allow the Receiving Party to acquire information similar to the Confidential Information from a third party?
. Further, the Parties agree that the Recipient shall use any and all Confidential Information received from the Discloser solely for the Purpose and no other use may be made without the Discloser’s prior written consent, which shall be granted or with-held in its sole discretion.
; (3) will, if reasonably required, cause each of the Recipient’s employees, officers, agents, affiliates and attorneys who have access to the Confidential Information to sign a confidentiality and non-disclosure agreement, for the benefit of the Discloser, containing terms at least as stringent as those contained in this Agreement, and, in any event, advise all of its employees, officers, agents, and affiliates of the confidential and proprietary nature of the Confidential Information
4. Nondisclosure. Except as descri

In [ ]:
# # generate responses for all the queries in the dataset

# def generate_full_response(response_generator: ResponseGeneration, size=10, JSON_CoT=True):
#     qna_context_list = []

#     # use tqdm here! 
#     for i in range(0, size):
#         query, contexts = get_dataset_from_json(index=i)
#         output = response_generator.graph.invoke({"question": query, "context": [context["chunk"] for context in contexts]})
        
#         user_input = query
#         retrieved_contexts = [context["chunk"] for context in contexts]

#         if JSON_CoT:
#             # Clean the output to ensure it's a valid JSON string
#             clean_answer = output["answer"].content.replace("\n", "").replace("\r", "").replace("\t", "").strip()
#             try:
#                 thought = json.loads(clean_answer)["thought"]
#                 response = json.loads(clean_answer)["answer"]
#                 qna_context_list.append([user_input, thought, response, retrieved_contexts])
#             except json.JSONDecodeError:
#                 print(f"Index: {i}, Output: {output['answer'].content}")
            
#             dataset_df = pd.DataFrame(qna_context_list, columns=["user_input", "thought", "response", "retrieved_contexts"])

#         else:
#             response = output["answer"].content
#             qna_context_list.append([user_input, response, retrieved_contexts])

#             dataset_df = pd.DataFrame(qna_context_list, columns=["user_input", "response", "retrieved_contexts"])
    
#     return dataset_df

In [50]:
# generate responses for all the queries in the dataset

def generate_full_response(response_generator: ResponseGeneration, size=10, JSON_CoT=True):
    qna_context_list = []
    
    for i in range(0, size):
        query, contexts = get_dataset_from_json(index=i)
        output = response_generator.graph.invoke({"question": query, "context": [context["chunk"] for context in contexts]})
        
        user_input = query
        retrieved_contexts = [context["chunk"] for context in contexts]

        if JSON_CoT:
            # Prepare the raw response for later parsing
            raw_response = output["answer"].content.strip()
            qna_context_list.append([user_input, raw_response, retrieved_contexts])
            
            # Optionally, write the raw response to a file for later parsing
            # with open(f'output_{i}.json', 'w') as f:
            #     json.dump({"raw_response": raw_response}, f)

            dataset_df = pd.DataFrame(qna_context_list, columns=["user_input", "raw_response", "retrieved_contexts"])

        else:
            response = output["answer"].content
            qna_context_list.append([user_input, response, retrieved_contexts])

            dataset_df = pd.DataFrame(qna_context_list, columns=["user_input", "response", "retrieved_contexts"])
    
    return dataset_df

In [51]:
query_answer_zeroshot_CoT = generate_full_response(zeroshot_CoT_response_generation, 194)


In [52]:
# Save the raw responses to a JSON file for backup
query_answer_zeroshot_CoT.to_json('query_answer_zeroshot_CoT_backup.json')


In [53]:
# Load the backup data
df = pd.read_json('query_answer_zeroshot_CoT_backup.json')


In [55]:
def process_raw_response_df(df):
    processed_data = []
    
    for idx, row in df.iterrows():
        user_input = row['user_input']
        raw_response = row['raw_response']
        retrieved_contexts = row['retrieved_contexts']
        
        # Clean the raw response string
        clean_response = raw_response.replace("\n", "").replace("\r", "").replace("\t", "").strip()
        
        try:
            # Try to parse as JSON directly first
            if clean_response.startswith("```json"):
                # Remove ```json and ``` markers
                json_str = clean_response.replace("```json", "").replace("```", "").strip()
                json_data = json.loads(json_str)
            else:
                json_data = json.loads(clean_response)
            thought = json_data["thought"]
            response = json_data["answer"]
            
        except json.JSONDecodeError:
            # If direct parsing fails, try to extract JSON structure
            try:
                # Find the last occurrence of {"thought"
                thought_start = clean_response.rindex('{"thought"')
                # Find the matching closing brace
                brace_count = 0
                for i in range(thought_start, len(clean_response)):
                    if clean_response[i] == '{':
                        brace_count += 1
                    elif clean_response[i] == '}':
                        brace_count -= 1
                        if brace_count == 0:
                            json_str = clean_response[thought_start:i+1]
                            break
                
                json_data = json.loads(json_str)
                thought = json_data["thought"]
                response = json_data["answer"]
                
            except (ValueError, json.JSONDecodeError, KeyError) as e:
                print(f"Failed to parse JSON at index {idx}")
                print(f"Raw response: {raw_response}")
                # Skip this row or add placeholder values
                thought = "ERROR: Failed to parse thought"
                response = "ERROR: Failed to parse response"
        
        processed_data.append([user_input, thought, response, retrieved_contexts])
    
    return pd.DataFrame(processed_data, columns=["user_input", "thought", "response", "retrieved_contexts"])

# Process the raw responses into structured format
cot_processed_df = process_raw_response_df(df)


In [56]:
display(cot_processed_df.columns)

Index(['user_input', 'thought', 'response', 'retrieved_contexts'], dtype='object')

In [57]:
cot_processed_df.to_json("./data/json_output/contractNLI_RG_zeroshot_CoT_improve1.json", orient="records", indent=4)

In [58]:
len(cot_processed_df)

193